In [3]:
import json

with open('claims_entities_spacy.json','r',encoding='UTF-8') as f:
    claiments = json.load(f)

In [4]:
import json

with open('tweets_entities_spacy.json','r',encoding='UTF-8') as f:
    tweetents = json.load(f)

In [5]:
import pandas as pd

tweets = pd.read_csv('birdwatch_tweets.csv')

In [6]:
claims = pd.read_csv('claim_reviews_en.csv')

In [7]:
tweetentities = []
for tweet in tweetents:
    entitylist = [ent[0] for ent in tweet]
    tweetentities.append(entitylist)

In [8]:
for i in range(len(tweetentities)):
    if None in tweetentities[i]:
        tweetentities[i] = [ent for ent in tweetentities[i] if ent is not None]

In [9]:
tweet_entities = []
tweet_edges = []

for sent, ents in zip(tweets.tweet,tweetentities):
    for ent in ents:
        tweet_entities.append(ent)
        tweet_edges.append((sent,ent))

In [10]:
claimentities = []
for claim in claiments:
    entitylist = [ent[0] for ent in claim]
    claimentities.append(entitylist)

In [11]:
for i in range(len(claimentities)):
    if None in claimentities[i]:
        claimentities[i] = [ent for ent in claimentities[i] if ent is not None]

In [12]:
claim_entities = []
claim_edges = []

for sent, ents in zip(claims.claim_text,claimentities):
    for ent in ents:
        claim_entities.append(ent)
        claim_edges.append((sent,ent))

In [13]:
edges = tweet_edges
edges.extend(claim_edges)

In [14]:
import networkx as nx
G = nx.Graph()

In [15]:
G.add_nodes_from(tweets.tweet)
G.add_nodes_from(tweet_entities)
G.add_nodes_from(claim_entities)
G.add_nodes_from(claims.claim_text)

In [16]:
G.add_edges_from(edges)

In [17]:
mapping = {}

for sent in claims.claim_text:
    mapping[sent] = {"type": "claim"}
    
for ent in claim_entities:
    mapping[ent] = {"type": "claimentity"}

for sent in tweets.tweet:
    mapping[sent] = {"type": "tweet"}

for ent in tweet_entities:
    mapping[ent] = {"type": "tweetentity"}
    

nx.set_node_attributes(G, mapping)

In [ ]:
from node2vec import Node2Vec

In [22]:
node2vec = Node2Vec(G, dimensions=64, walk_length=10, num_walks=20, workers=1)  # Use temp_folder for big graphs

Computing transition probabilities:   0%|          | 0/73410 [00:00<?, ?it/s]

Generating walks (CPU: 1): 100%|███████████████████████████████████████████████████████| 20/20 [03:46<00:00, 11.32s/it]


In [23]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [25]:
# Save embeddings for later use
model.wv.save_word2vec_format('all_embeddings_corrected.bin')

# Save model for later use
model.save('node2vec_model_corrected.bin')

In [1]:
from gensim.models import Word2Vec
model = Word2Vec.load('node2vec_model_corrected.bin')

In [24]:
r = model.wv.most_similar("trump is right, he didn't start a new war (def tried w/ iran) but what's more impressive is that trump killed 400k americans without even starting a new war!", topn=len(G.nodes))

In [26]:
r_sent = []

for bla in r:
    try:
        if G.nodes[bla[0]]["type"] == "claim":
            r_sent.append(bla)
    except:
        continue

In [27]:
r_sent

[('"Iran just test-fired a Ballistic Missile capable of reaching Israel."',
  0.7017028331756592),
 ('Iran objavio video atentata na Trumpa', 0.6910170912742615),
 ('Says Rep. Walter Jones “caved to Obama” and “empowered Obama to cut a deal with Iran.”',
  0.6794388294219971),
 ('The Jamkaran mosque in Iran raised a red flag over its dome for the first time in history.',
  0.6514610648155212),
 ('The decision for the United States to buy heavy water from Iran "will directly subsidize Iran’s nuclear program."',
  0.6495444774627686),
 ('The Obama administration gave Iran "$400 million in ransom payment cash."',
  0.6388788223266602),
 ('Says “last week, Israel published intelligence documents, long concealed by Iran, conclusively showing the Iranian regime and its history of pursuing nuclear weapons.”',
  0.6380752921104431),
 ('Says President Obama’s deal "allows Iran to produce a nuclear weapon."',
  0.635999858379364),
 ('On the Iran nuclear deal', 0.6320854425430298),
 ('"In seven y